# PyCity Schools Analysis

## Analysis is below


In [129]:
#Dependencies and Setup
import pandas as pd
from pathlib import Path

#Files to Load 
school_data_to_load= Path("Resources/schools_complete.csv")
student_data_to_load= Path("Resources/students_complete.csv")

#Files to read and store into Pandas
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

#Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

school_data_complete.head()



,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [130]:
#Total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count

15

In [131]:
#Total number of students 
student_count = school_data_complete["student_name"].count()
student_count

39170

In [132]:
#Calculation of the average math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [133]:
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [134]:
#Percentage of students who passed math (math scores greater than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count)*100
passing_math_percentage

74.9808526933878

In [135]:
#Percentage of students who passed reading
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count)*100
passing_reading_percentage

85.80546336482001

In [136]:
#Percentage of students who passed math and reading
passing_math_reading_count = school_data_complete[(school_data_complete["math_score"]>=70) & (school_data_complete["reading_score"]>=70)].count()["student_name"]
passing_math_reading_percentage = passing_math_reading_count / float(student_count)*100
passing_math_reading_percentage

65.17232575950983

In [137]:
#Total Budget
list_budget = school_data_complete["budget"].unique()
total_budget = list_budget.sum()

#Create a high-level snapshot of the district's key metrics in a Dataframe
district_summary =pd.DataFrame([{"Total School":school_count,"Total Students":student_count,"Total Budget":total_budget, "Average Math Score":average_math_score, 
                     "Average Reading Score":average_reading_score,"% Passing Math":passing_math_percentage, "% Passing Reading":passing_reading_percentage,
                     "% Overall Passing" : passing_math_reading_percentage}])
#Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [138]:
# The type per school from school data
school_data_complete_df=school_data_complete.groupby(["school_name"])
school_data_complete_df.sum().head()
school_types = school_data.set_index(["school_name"])["type"]
# school_types 

In [139]:
#Calculate the total student per school from school_data
per_school_counts = school_data.set_index(["school_name"])["size"]


In [140]:
#Calculate the total school budget and per capita spending per school from school data
per_school_budget = school_data.set_index(["school_name"])["budget"]
per_school_capita = per_school_budget/per_school_counts
per_school_capita_df = per_school_capita.to_frame()

In [141]:
# Caclulate the average test scores per school from school_data_complete
per_school_math = school_data_complete.groupby(["school_name"])["math_score"].mean()
per_school_math_df = per_school_math.to_frame()
per_school_math_df = per_school_math_df.reset_index()
per_school_math_df = per_school_math_df.set_index("school_name")

per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()
per_school_reading_df = per_school_reading.to_frame()
per_school_reading_df = per_school_reading_df.reset_index()
per_school_reading_df = per_school_reading_df.set_index("school_name")


In [142]:
# Calculate the number of students per school with math and with reading scores of 70 or higher from school_data_complete
students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)].groupby(["school_name"]).size()

students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)].groupby(["school_name"]).size()


In [143]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70) ].groupby(["school_name"]).size()


In [144]:
# Use the provided code to calculate the passing rates
per_school_passing_math = students_passing_math / per_school_counts * 100
per_school_passing_math_df = per_school_passing_math.to_frame()  
per_school_passing_reading = students_passing_reading / per_school_counts * 100
per_school_passing_reading_df = per_school_passing_reading.to_frame()
overall_passing_rate = students_passing_math_and_reading / per_school_counts * 100
overall_passing_rate_df = overall_passing_rate.to_frame() 



In [145]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
                             

per_school_summary1 = pd.merge(school_types,per_school_counts, how='left',left_index=True,right_index=True)
per_school_summary2 = pd.merge(per_school_summary1,per_school_budget,how='left',left_index=True,right_index=True)
per_school_summary3 = pd.merge(per_school_summary2,per_school_capita_df,how='left',left_index=True,right_index=True)
per_school_summary4 = pd.merge(per_school_summary3,per_school_math_df,how='left',left_index=True,right_index=True)
per_school_summary5 = pd.merge(per_school_summary4,per_school_reading_df,how='left',left_index=True,right_index=True)
per_school_summary6 = pd.merge(per_school_summary5,per_school_passing_math_df,how='left',left_index=True,right_index=True)
per_school_summary7 = pd.merge(per_school_summary6,per_school_passing_reading_df,how='left',left_index=True,right_index=True)
per_school_summary8 = pd.merge(per_school_summary7,overall_passing_rate_df , how='left',left_index=True,right_index=True, 
                               suffixes=('_left', '_right'))
renamed_per_school_summary = per_school_summary8.rename(columns={"type":"School Type","size":"Total Students","budget":"Total School Budget", 
                                                                 "0_x": "Per Student Budget", "math_score":"Average Math Score","reading_score":"Average Reading Score",
                                                                 "0_y":"% Passing Math", "0_left":"% Passing Reading", "0_right":"% Overall Passing"})
#Formatting
renamed_per_school_summary["Total School Budget"] = renamed_per_school_summary["Total School Budget"].map("${:,.2f}".format)
renamed_per_school_summary["Per Student Budget"] = renamed_per_school_summary["Per Student Budget"].map("${:,.2f}".format)

renamed_per_school_summary  

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166


# Highest-Performing Schools (by % Overall Passing)

In [146]:
#Sort the shcools by % Overall Passing in descending order and display the top 5 rows
top_schools = renamed_per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


# Bottom Performing Schools (By % Overall Passing)

In [147]:
#Sort the shcools by % Overall Passing in ascending order and display the top 5 rows
top_schools = renamed_per_school_summary.sort_values("% Overall Passing", ascending=True)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


# Math Scores by Grade

In [148]:
#Use the code provided to separate the data by grade 
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

#Group by 'school_name' and take the mean of the 'math_score' column for each.
ninth_grader_math_scores = ninth_graders.groupby(["school_name"])["math_score"].mean()
ninth_grader_math_scores_df = ninth_grader_math_scores.to_frame()
tenth_grader_math_scores = tenth_graders.groupby(["school_name"])["math_score"].mean()
tenth_grader_math_scores_df = tenth_grader_math_scores.to_frame()
elevent_grader_math_scores = eleventh_graders.groupby(["school_name"])["math_score"].mean()
elevent_grader_math_scores_df = elevent_grader_math_scores.to_frame()
twelfth_grader_math_scores = twelfth_graders.groupby(["school_name"])["math_score"].mean()
twelfth_grader_math_scores_df = twelfth_grader_math_scores.to_frame()

#Combine each of the scores above into single Df 
math_scores_by_grade1 = pd.merge(ninth_grader_math_scores_df,tenth_grader_math_scores_df, how='left',left_index=True,right_index=True)
math_scores_by_grade2 = pd.merge(math_scores_by_grade1,elevent_grader_math_scores_df, how='left',left_index=True,right_index=True)
math_scores_by_grade3 = pd.merge(math_scores_by_grade2,twelfth_grader_math_scores_df, how='left',left_index=True,right_index=True, suffixes=('_left', '_right'))
math_scores_by_grade = math_scores_by_grade3.rename(columns={"math_score_x":"9th", "math_score_y":"10th", "math_score_left":"11th", "math_score_right":"12th"})
math_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Score by Grade

In [149]:
#Use the code provided to separate the data by grade 
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

#Group by 'school_name' and take the mean of the 'reading_score' column for each.
ninth_grader_reading_scores = ninth_graders.groupby(["school_name"])["reading_score"].mean()
ninth_grader_reading_scores_df = ninth_grader_reading_scores.to_frame()
tenth_grader_reading_scores = tenth_graders.groupby(["school_name"])["reading_score"].mean()
tenth_grader_reading_scores_df = tenth_grader_reading_scores.to_frame()
elevent_grader_reading_scores = eleventh_graders.groupby(["school_name"])["reading_score"].mean()
elevent_grader_reading_scores_df = elevent_grader_reading_scores.to_frame()
twelfth_grader_reading_scores = twelfth_graders.groupby(["school_name"])["reading_score"].mean()
twelfth_grader_reading_scores_df = twelfth_grader_reading_scores.to_frame()

#Combine each of the scores above into single Df 
reading_scores_by_grade1 = pd.merge(ninth_grader_reading_scores_df,tenth_grader_reading_scores_df, how='left',left_index=True,right_index=True)
reading_scores_by_grade2 = pd.merge(reading_scores_by_grade1,elevent_grader_reading_scores_df, how='left',left_index=True,right_index=True)
reading_scores_by_grade3 = pd.merge(reading_scores_by_grade2,twelfth_grader_reading_scores_df, how='left',left_index=True,right_index=True, suffixes=('_left', '_right'))
reading_scores_by_grade = reading_scores_by_grade3.rename(columns={"reading_score_x":"9th", "reading_score_y":"10th", "reading_score_left":"11th", "reading_score_right":"12th"})
reading_scores_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by School Spending

In [150]:
#Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [151]:
# Create a copy of the school summary since it has the "Per Student Budget"
school_spending_df = renamed_per_school_summary.copy()

school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace('$', '')
school_spending_df["Per Student Budget"] = pd.to_numeric(school_spending_df["Per Student Budget"])
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels=labels, include_lowest=True)
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].map("${:,.2f}".format)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567,<$585
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585


In [152]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=False)["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=False)["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"],observed=False)["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=False)["% Overall Passing"].mean()
 

In [153]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score":spending_math_scores, "Average Reading Score":spending_reading_scores,
                                 "% Passing Math": spending_passing_math, "% Passing Reading": spending_passing_reading, 
                                 "%Overall Passing": overall_passing_spending })
# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


# Scores by School Size

In [154]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [155]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

renamed_per_school_summary["School Size"] = pd.cut(renamed_per_school_summary["Total Students"], size_bins, labels=labels, include_lowest=True)
renamed_per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
school_name,,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,89.892107,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)


In [156]:
# Calculate averages for the desired columns.
size_math_scores = renamed_per_school_summary.groupby(["School Size"], observed=False)["Average Math Score"].mean()
size_reading_scores = renamed_per_school_summary.groupby(["School Size"], observed=False)["Average Reading Score"].mean()
size_passing_math = renamed_per_school_summary.groupby(["School Size"], observed=False)["% Passing Math"].mean()
size_passing_reading = renamed_per_school_summary.groupby(["School Size"], observed=False)["% Passing Reading"].mean()
size_overall_passing = renamed_per_school_summary.groupby(["School Size"], observed=False)["% Overall Passing"].mean()


In [157]:
# Create a DataFrame called 'size_summary' that breaks down school performance based on school size (small, medium, large).
# Use the scores above to create a new DataFrame called 'size_summary'
size_summary = pd.DataFrame({"Average Math Score":size_math_scores, "Average Reading Score":size_reading_scores, "% Passing Math":size_passing_math,
                             "% Passing Reaading":size_passing_math, "% Overall Passing":size_overall_passing})
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reaading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,93.550225,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,93.599695,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,69.963361,58.286003


# Scores by School Type

In [158]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_scores_by_type = renamed_per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = renamed_per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = renamed_per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = renamed_per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = renamed_per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()


In [159]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({"Average Math Score":average_math_scores_by_type, "Average Reading Score":average_reading_score_by_type,
                                "% Passing Math":average_percent_passing_math_by_type, "% Passing Reading":average_percent_passing_reading_by_type,
                                "% Overall Passing":average_percent_overall_passing_by_type})
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
